This notebook is developed using the `Python 3 (TensorFlow 2.3 Python 3.7 CPU Optimized)` kernel on an `ml.t3.medium` instance.

In [ ]:
import sagemaker
import json
import boto3

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
prefix = 'sagemaker-studio-book/chapter07'

In [ ]:
import numpy as np
import os
from tensorflow.python.keras.datasets import imdb

In [ ]:
csv_test_dir_prefix = 'imdb_data/test'
csv_test_filename = 'test.csv'

x_test = np.loadtxt(f'{csv_test_dir_prefix}/{csv_test_filename}', 
                    delimiter=',', dtype='int')

print('x_test shape:', x_test.shape)

max_features = 20000
maxlen = 400

_ , (_, y_test) = imdb.load_data(num_words=max_features)

In [ ]:
from sagemaker.tensorflow import TensorFlow

# insert the training job name from chapter05/02-tensorflow_sentiment_analysis.ipynb
training_job_name= '<your-training-job-name>' # e.g. imdb-tf-2022-02-18-05-31-26

estimator = TensorFlow.attach(training_job_name)

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, 
                             instance_type='ml.c5.xlarge')

In [ ]:
predictor.endpoint_name

In [ ]:
data_index=199

In [ ]:
prediction=predictor.predict(x_test[data_index])
print(prediction)

In [ ]:
def get_sentiment(score):
    return 'positive' if score > 0.5 else 'negative' 

import re

regex = re.compile(r'^[\?\s]+')
word_index = imdb.get_word_index()

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
first_decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') 
                                 for i in x_test[data_index]])
regex.sub('', first_decoded_review)

In [ ]:
print(f'Labeled sentiment for this review is {get_sentiment(y_test[data_index])}')
print(f'Predicted sentiment is {get_sentiment(prediction["predictions"][0][0])}')

In [ ]:
predictor.predict(x_test[:5000]) # this would throw an error due to large volume

In [ ]:
sagemaker_client = sess.boto_session.client('sagemaker')
autoscaling_client = sess.boto_session.client('application-autoscaling')

endpoint_name = predictor.endpoint_name
response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
print(response)

In [ ]:
# application autoscaling references the endpoint using string below
resource_id=f'endpoint/{endpoint_name}/variant/AllTraffic' 
response = autoscaling_client.register_scalable_target(
    ServiceNamespace='sagemaker',
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    MinCapacity=1,
    MaxCapacity=4
)

response = autoscaling_client.put_scaling_policy(
    PolicyName='Invocations-ScalingPolicy',
    ServiceNamespace='sagemaker', # The namespace of the AWS service that provides the resource. 
    ResourceId=resource_id, # Endpoint name 
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    PolicyType='TargetTrackingScaling', # Other options are 'StepScaling'|'TargetTrackingScaling'
    TargetTrackingScalingPolicyConfiguration={
        'TargetValue': 4000.0, # The target value for the metric below.
        'PredefinedMetricSpecification': {
            'PredefinedMetricType': 'SageMakerVariantInvocationsPerInstance', 
        },
        'ScaleInCooldown': 600, 
        'ScaleOutCooldown': 300,
        'DisableScaleIn': False # If true, scale-in is disabled.
    }
)

In [ ]:
response = autoscaling_client.describe_scaling_policies(
    ServiceNamespace='sagemaker'
)

for i in response['ScalingPolicies']:
    print('')
    print(i['PolicyName'])
    print('')
    if('TargetTrackingScalingPolicyConfiguration' in i):
        print(i['TargetTrackingScalingPolicyConfiguration']) 
    else:
        print(i['StepScalingPolicyConfiguration'])
    print('')

You reach the end of this example. Typically it's recommended to delete the endpoint to stop incurring cost. However, the endpoint deployed in this notebook will be used again in [04-load_testing.ipynb](./04-load_testing.ipynb), so deletion is optional when you will soon advance to [04-load_testing.ipynb](./04-load_testing.ipynb).

If you decide to use the endpoint for [04-load_testing.ipynb](./04-load_testing.ipynb), you need the endpoint name later.

In [ ]:
print(endpoint_name)

In [ ]:
# predictor.delete_endpoint()